In [1]:
import numpy as np
import tensorflow as tf
import re
import time

/home/akshaydv/anaconda3/envs/tensorflow/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
lines = open('movie_lines.txt', encoding = 'utf-8', errors = 'ignore').read().split('\n')
conversations = open('movie_conversations.txt', encoding = 'utf-8', errors = 'ignore').read().split('\n')

In [3]:
id2line = {}
for line in lines:
    _line = line.split(' +++$+++ ')
    if len(_line) == 5:
        id2line[_line[0]] = _line[4]

In [4]:
conversations_ids = []
for conversation in conversations[:-1]:
    _conversation = conversation.split(' +++$+++ ')[-1][1:-1].replace("'", "").replace(" ", "")
    conversations_ids.append(_conversation.split(','))

In [5]:
questions = []
answers = []
for conversation in conversations_ids:
    for i in range(len(conversation) - 1):
        questions.append(id2line[conversation[i]])
        answers.append(id2line[conversation[i+1]])

In [6]:
def clean_text(text):
    text = text.lower()
    text = re.sub(r"i'm", "i am", text)
    text = re.sub(r"he's", "he is", text)
    text = re.sub(r"she's", "she is", text)
    text = re.sub(r"that's", "that is", text)
    text = re.sub(r"what's", "what is", text)
    text = re.sub(r"where's", "where is", text)
    text = re.sub(r"\'ll", " will", text)
    text = re.sub(r"\'ve", " have", text)
    text = re.sub(r"\'re", " are", text)
    text = re.sub(r"\'d", " would", text)
    text = re.sub(r"won't", "will not", text)
    text = re.sub(r"can't", "cannot", text)
    text = re.sub(r"[-()\"#/@;:<>{}+=~|.?,]", "", text)
    return text

In [7]:
clean_questions = []
for question in questions:
    clean_questions.append(clean_text(question))

clean_answers = []
for answer in answers:
    clean_answers.append(clean_text(answer))

In [10]:
word2count = {}
for question in clean_questions:
    for word in question.split():
        if word not in word2count:
            word2count[word] = 1
        else:
            word2count[word] += 1
for answer in clean_answers:
    for word in answer.split():
        if word not in word2count:
            word2count[word] = 1
        else:
            word2count[word] += 1

In [11]:
threshold_questions = 20
questionswords2int = {}
word_number = 0
for word, count in word2count.items():
    if count >= threshold_questions:
        questionswords2int[word] = word_number
        word_number += 1
threshold_answers = 20
answerswords2int = {}
word_number = 0
for word, count in word2count.items():
    if count >= threshold_answers:
        answerswords2int[word] = word_number
        word_number += 1

In [12]:
tokens = ['<PAD>', '<EOS>', '<OUT>', '<SOS>']
for token in tokens:
    questionswords2int[token] = len(questionswords2int) + 1
for token in tokens:
    answerswords2int[token] = len(answerswords2int) + 1

In [13]:
answersints2word = {w_i: w for w, w_i in answerswords2int.items()}

In [14]:
for i in range(len(clean_answers)):
    clean_answers[i] += ' <EOS>'

In [15]:
questions_into_int = []
for question in clean_questions:
    ints = []
    for word in question.split():
        if word not in questionswords2int:
            ints.append(questionswords2int['<OUT>'])
        else:
            ints.append(questionswords2int[word])
    questions_into_int.append(ints)
answers_into_int = []
for answer in clean_answers:
    ints = []
    for word in answer.split():
        if word not in answerswords2int:
            ints.append(answerswords2int['<OUT>'])
        else:
            ints.append(answerswords2int[word])
    answers_into_int.append(ints)

In [16]:
sorted_clean_questions = []
sorted_clean_answers = []
for length in range(1, 25 + 1):
    for i in enumerate(questions_into_int):
        if len(i[1]) == length:
            sorted_clean_questions.append(questions_into_int[i[0]])
            sorted_clean_answers.append(answers_into_int[i[0]])

In [27]:
sorted_clean_answers

[[5000,
  4600,
  8555,
  3284,
  3023,
  1747,
  6414,
  5000,
  5791,
  6492,
  2643,
  5632,
  8824,
  5914,
  6492,
  5645,
  7329,
  6884,
  3023,
  7228,
  7947,
  5244,
  732,
  3411,
  8823],
 [8824,
  870,
  732,
  6541,
  3020,
  2192,
  1673,
  7630,
  2283,
  732,
  6757,
  2808,
  7650,
  6450,
  1583,
  1210,
  474,
  1226,
  732,
  7972,
  8658,
  6492,
  1583,
  7492,
  6826,
  8823],
 [5908, 8823],
 [3715, 1226, 2793, 8682, 8433, 3068, 6022, 8823],
 [2179, 6872, 3943, 1347, 5709, 8823],
 [1346, 2179, 4992, 7879, 2035, 3020, 7170, 7646, 3020, 4909, 8823],
 [1846, 6920, 8823],
 [2179, 456, 6624, 4141, 4863, 1583, 8823],
 [2022, 8823],
 [3847, 2179, 167, 7661, 4726, 2643, 4731, 3284, 8823],
 [7329,
  1910,
  3229,
  3023,
  8292,
  7972,
  2643,
  3252,
  8353,
  6414,
  3187,
  5000,
  8824,
  2757,
  1765,
  4231,
  8823],
 [2763, 8824, 4123, 2643, 1896, 8823],
 [7596, 1210, 1226, 2643, 6741, 4416, 8823],
 [3278,
  4236,
  3023,
  2384,
  3023,
  3847,
  6344,
  7009,
 